## Chapter 3: Getting Started with ASP.NET Core MVC

Now that you understand C# fundamentals and how HTTP works, it's time to build your first web application using ASP.NET Core MVC. MVC (Model-View-Controller) is a proven architectural pattern that separates concerns, making your code organized, testable, and maintainable. In this chapter, you’ll create a new MVC project, explore its structure, learn how routing maps URLs to actions, dive deep into controllers, and build views using the Razor syntax. By the end, you’ll have a working web application that displays dynamic content.

### 3.1 The MVC Architectural Pattern (Model-View-Controller)

MVC divides your application into three interconnected components:

- **Model**: Represents the data and business logic. It knows nothing about the user interface. In a web app, models are typically plain C# classes that hold data (e.g., a `Product` class) and may include validation rules or business methods.
- **View**: Responsible for presenting data to the user. In ASP.NET Core, views are `.cshtml` files that mix HTML with C# code using Razor syntax. Views are purely presentational; they should not contain complex logic.
- **Controller**: Handles user input and interactions. It receives HTTP requests, works with the model (e.g., fetches data from a database), selects a view to render, and returns the response. Controllers are the traffic cops of your application.

**How they interact:**
1. The browser sends an HTTP request (e.g., `GET /products`).
2. The routing system maps the URL to a specific controller action (e.g., `ProductsController.Index()`).
3. The controller action processes the request: it might fetch data from the model (e.g., query a database via Entity Framework Core), perform business logic, and prepare data for the view.
4. The controller returns a view, passing the model data to it.
5. The view generates HTML using the model data and sends it back to the browser.

This separation allows multiple developers to work on different parts, makes unit testing easier (you can test controllers without views), and promotes reusability.

---

### 3.2 Creating Your First MVC Project

Let’s create a new ASP.NET Core MVC project. You can use either the .NET CLI or Visual Studio.

#### Using the .NET CLI
Open a terminal and run:
```bash
dotnet new mvc -n MyFirstMvcApp
cd MyFirstMvcApp
dotnet run
```

#### Using Visual Studio 2022
1. File → New → Project.
2. Select "ASP.NET Core Web App (Model-View-Controller)".
3. Name it `MyFirstMvcApp` and choose a location.
4. Select .NET 8.0 (or latest LTS) and keep default options.
5. Click Create.

After creation, you'll see a solution with several folders and files:

```
MyFirstMvcApp/
├── Controllers/         # Controllers go here
├── Views/               # Views, organized by controller
│   ├── Home/            # Views for HomeController
│   ├── Shared/          # Shared views (layout, partials)
│   └── _ViewStart.cshtml
├── Models/              # Model classes
├── wwwroot/             # Static files (CSS, JS, images)
├── appsettings.json     # Configuration
├── Program.cs           # Application startup
└── MyFirstMvcApp.csproj # Project file
```

**Run the application:**
- If using CLI, navigate to the project folder and run `dotnet run`. Open `https://localhost:5001` (or the URL shown) in your browser.
- If using Visual Studio, press F5.

You'll see a default home page with a navigation bar. This template includes sample pages for Home, Privacy, etc., giving you a starting point.

---

### 3.3 Routing: Mapping URLs to Controllers and Actions

Routing is the mechanism that matches incoming URLs to controller actions. ASP.NET Core supports two types of routing: conventional routing and attribute routing.

#### Conventional Routing

Conventional routing uses a route template defined in `Program.cs`. The default template looks like this:

```csharp
app.MapControllerRoute(
    name: "default",
    pattern: "{controller=Home}/{action=Index}/{id?}");
```

Let's break down the pattern:
- `{controller=Home}`: The first segment is the controller name. If not provided, it defaults to `Home`.
- `{action=Index}`: The second segment is the action name. Defaults to `Index`.
- `{id?}`: The third segment is an optional `id` parameter. The `?` makes it optional.

So, a request to `/products/details/5` would map to `ProductsController` with action `Details` and `id = 5`. A request to `/` maps to `HomeController.Index()`.

Conventional routing is centralized and consistent. It works well for traditional web applications with many pages.

#### Attribute Routing

Attribute routing gives you more control by placing routes directly on controllers and actions using attributes. This is the preferred approach for Web APIs and is also commonly used in MVC applications for specific routes.

```csharp
[Route("products")]
public class ProductsController : Controller
{
    [Route("")]              // matches /products
    [Route("index")]         // matches /products/index
    public IActionResult Index() { ... }

    [Route("details/{id}")]  // matches /products/details/5
    public IActionResult Details(int id) { ... }

    [Route("create")]
    public IActionResult Create() { ... }
}
```

You can combine both routing styles in the same application. To enable attribute routing, you typically call `app.MapControllers()` (for API controllers) or just rely on the default conventional route. Attribute routing takes precedence.

#### Route Parameters and Constraints

You can add constraints to route parameters to enforce types:

```csharp
[Route("products/details/{id:int}")]  // id must be an integer
```

Common constraints: `int`, `bool`, `datetime`, `guid`, `min(5)`, `max(10)`, `regex(...)`.

**Example in a controller action:**

```csharp
[HttpGet("products/details/{id:int}")]
public IActionResult Details(int id)
{
    // id is guaranteed to be an integer
    var product = _productService.GetById(id);
    if (product == null)
        return NotFound();
    return View(product);
}
```

**Why routing matters:** It decouples your URL structure from your code organization. You can change URLs without renaming controllers, and you can create user-friendly, SEO-optimized URLs.

---

### 3.4 Controllers in Depth: Handling Requests and Returning Results

A controller is a class that derives from `Microsoft.AspNetCore.Mvc.Controller`. It contains public methods called **actions**. Each action corresponds to an endpoint.

#### Creating a Simple Controller

Add a new controller named `ProductsController` inside the `Controllers` folder:

```csharp
using Microsoft.AspNetCore.Mvc;

namespace MyFirstMvcApp.Controllers
{
    public class ProductsController : Controller
    {
        public IActionResult Index()
        {
            return View();
        }

        public IActionResult Details(int id)
        {
            ViewBag.ProductId = id;
            return View();
        }
    }
}
```

Now create corresponding views:
- `Views/Products/Index.cshtml`
- `Views/Products/Details.cshtml`

You can generate these by right-clicking the action method in Visual Studio and selecting "Add View".

#### Action Results

Actions return implementations of `IActionResult`. Common results include:

- `View()` – Renders a view (default is same name as action). Can pass a model.
- `PartialView()` – Renders a partial view.
- `Json()` – Returns JSON data.
- `Content()` – Returns plain text.
- `Redirect()` / `RedirectToAction()` – Redirects to another URL or action.
- `NotFound()` – Returns HTTP 404.
- `BadRequest()` – Returns HTTP 400.
- `Ok()` – Returns HTTP 200 (useful in API controllers).
- `StatusCode()` – Returns any HTTP status code.

**Example with various results:**

```csharp
public class TestController : Controller
{
    public IActionResult Text() => Content("Hello, world!");

    public IActionResult JsonData() => Json(new { Name = "Sample", Price = 19.99 });

    public IActionResult RedirectExample() => Redirect("https://example.com");

    public IActionResult GoToIndex() => RedirectToAction("Index", "Home");

    public IActionResult NotFoundPage() => NotFound();

    public IActionResult Error() => StatusCode(500);
}
```

#### Parameters in Actions

Actions can receive parameters from the request via:
- Route data (e.g., `{id}` in route template)
- Query string (`?page=2`)
- Form data (POST)
- Request body (JSON for APIs)

ASP.NET Core uses **model binding** to automatically map incoming data to action parameters.

```csharp
// URL: /products/search?category=electronics&page=2
public IActionResult Search(string category, int page = 1)
{
    // category = "electronics", page = 2
    return Content($"Searching {category}, page {page}");
}
```

For complex types, model binding creates an instance and populates its properties:

```csharp
public class SearchCriteria
{
    public string Category { get; set; }
    public int Page { get; set; }
    public int PageSize { get; set; }
}

public IActionResult Search(SearchCriteria criteria)
{
    // criteria.Category, criteria.Page, etc. are populated
    return View(criteria);
}
```

#### The Controller Base Class

The `Controller` base class provides many useful properties and methods:
- `Request` – access to the HTTP request.
- `Response` – manipulate the HTTP response.
- `HttpContext` – the full context.
- `User` – information about the current user.
- `ModelState` – validation state.
- `TempData` – dictionary for passing data between requests (uses cookies or session).
- `ViewData` / `ViewBag` – pass data from controller to view.

**Example using ViewBag:**

```csharp
public IActionResult Welcome(string name)
{
    ViewBag.UserName = name;
    ViewBag.CurrentTime = DateTime.Now;
    return View();
}
```

In the view (`Welcome.cshtml`), you can access:

```html
<h1>Welcome, @ViewBag.UserName!</h1>
<p>The current time is @ViewBag.CurrentTime</p>
```

`ViewBag` is a dynamic wrapper around `ViewData`. It's convenient but lacks compile-time checking. For strongly typed data, use a **view model** (discussed later).

---

### 3.5 Views and the Razor Syntax (`@`, `@if`, `@foreach`)

Views in ASP.NET Core are `.cshtml` files that combine HTML with C# code using Razor syntax. Razor is designed to be concise and expressive.

#### Razor Syntax Basics

- `@` transitions from HTML to C#. For example, `@DateTime.Now` outputs the current date and time.
- Code blocks are enclosed in `@{ ... }`. For example, you can declare variables or write multi-line logic.
- Control structures (`if`, `for`, `foreach`, `while`, `switch`) use a simplified syntax without braces (though you can add them for clarity).

**Simple example:**

```html
@{
    var message = "Hello, Razor!";
    var count = 5;
}

<h1>@message</h1>
<p>The count is @count.</p>

@if (count > 3)
{
    <p>Count is greater than 3.</p>
}
else
{
    <p>Count is not greater than 3.</p>
}

<ul>
@for (int i = 1; i <= count; i++)
{
    <li>Item @i</li>
}
</ul>
```

#### Using a Model in a View

Typically, you pass a strongly typed model from the controller to the view. This is done via the `View()` method:

```csharp
public IActionResult Details(int id)
{
    var product = new Product { Id = id, Name = "Laptop", Price = 999.99m };
    return View(product); // passes product as the model
}
```

In the view, you declare the model type at the top using `@model` directive:

```csharp
@model Product

<h2>@Model.Name</h2>
<p>Price: @Model.Price.ToString("C")</p>
<p>ID: @Model.Id</p>
```

`@model` (lowercase) specifies the type. `Model` (uppercase) is the instance.

#### Layout Pages and `_ViewStart.cshtml`

Most web applications share common elements across pages: header, footer, navigation menu. Layout pages provide this shared structure.

The default layout is `Views/Shared/_Layout.cshtml`. It contains the overall HTML structure and calls `@RenderBody()` where the page-specific content is inserted.

**Example `_Layout.cshtml` snippet:**

```html
<!DOCTYPE html>
<html>
<head>
    <title>@ViewBag.Title - My App</title>
    <link rel="stylesheet" href="~/css/site.css" />
</head>
<body>
    <header>...</header>
    <main>
        @RenderBody()
    </main>
    <footer>...</footer>
    @await RenderSectionAsync("Scripts", required: false)
</body>
</html>
```

`_ViewStart.cshtml` runs before every view. It typically sets the layout:

```csharp
@{
    Layout = "_Layout";
}
```

You can override the layout for a specific view by setting `Layout` inside the view.

#### Partial Views

Partial views are reusable pieces of Razor markup. They are stored in the `Shared` folder or in a controller-specific folder, usually with a leading underscore (e.g., `_ProductCard.cshtml`). Render a partial view using:

```html
<partial name="_ProductCard" model="someProduct" />
```

Or in code:

```csharp
@await Html.PartialAsync("_ProductCard", product)
```

#### View Components

View components are more powerful than partials. They have their own logic (similar to a mini-controller) and can be used for dynamic, self-contained widgets like a shopping cart summary or a login panel.

We'll cover view components in Chapter 4 (Building Interactive Web Pages with Razor), but here's a quick example:

```csharp
public class CartSummaryViewComponent : ViewComponent
{
    public IViewComponentResult Invoke()
    {
        var itemCount = GetCartItemCount(); // some logic
        return View(itemCount);
    }
}
```

Invoke in a view:

```html
@await Component.InvokeAsync("CartSummary")
```

#### HTML Helpers and Tag Helpers

ASP.NET Core provides two ways to generate HTML in views: HTML Helpers (C# methods) and Tag Helpers (HTML-friendly attributes). Tag Helpers are the modern, preferred approach because they look like standard HTML.

**Example of a Tag Helper for forms:**

```html
<form asp-action="Create" asp-controller="Products" method="post">
    <label asp-for="Name"></label>
    <input asp-for="Name" class="form-control" />
    <span asp-validation-for="Name"></span>
    
    <button type="submit">Save</button>
</form>
```

Tag Helpers like `asp-action`, `asp-for`, etc., are processed on the server and emit appropriate HTML. They make your views cleaner and more readable.

---

### 3.6 Putting It All Together: A Complete Example

Let's build a simple product listing and details page to solidify the concepts.

#### Step 1: Create a Model

In the `Models` folder, create `Product.cs`:

```csharp
namespace MyFirstMvcApp.Models
{
    public class Product
    {
        public int Id { get; set; }
        public string Name { get; set; }
        public decimal Price { get; set; }
        public string Description { get; set; }
    }
}
```

#### Step 2: Create a Controller

In `Controllers`, create `ProductsController.cs`:

```csharp
using Microsoft.AspNetCore.Mvc;
using MyFirstMvcApp.Models;
using System.Collections.Generic;
using System.Linq;

namespace MyFirstMvcApp.Controllers
{
    public class ProductsController : Controller
    {
        // Simulate a data store
        private static List<Product> _products = new List<Product>
        {
            new Product { Id = 1, Name = "Laptop", Price = 999.99m, Description = "High-performance laptop" },
            new Product { Id = 2, Name = "Mouse", Price = 19.99m, Description = "Wireless mouse" },
            new Product { Id = 3, Name = "Keyboard", Price = 49.99m, Description = "Mechanical keyboard" }
        };

        // GET: /products
        public IActionResult Index()
        {
            return View(_products);
        }

        // GET: /products/details/2
        public IActionResult Details(int id)
        {
            var product = _products.FirstOrDefault(p => p.Id == id);
            if (product == null)
            {
                return NotFound();
            }
            return View(product);
        }
    }
}
```

#### Step 3: Create Views

Create the `Views/Products` folder. Add `Index.cshtml`:

```html
@model IEnumerable<MyFirstMvcApp.Models.Product>

<h1>Product List</h1>

<table class="table">
    <thead>
        <tr>
            <th>ID</th>
            <th>Name</th>
            <th>Price</th>
            <th></th>
        </tr>
    </thead>
    <tbody>
    @foreach (var product in Model)
    {
        <tr>
            <td>@product.Id</td>
            <td>@product.Name</td>
            <td>@product.Price.ToString("C")</td>
            <td>
                <a asp-action="Details" asp-route-id="@product.Id">Details</a>
            </td>
        </tr>
    }
    </tbody>
</table>
```

Now add `Details.cshtml`:

```html
@model MyFirstMvcApp.Models.Product

<h1>Product Details</h1>

<div>
    <h4>@Model.Name</h4>
    <hr />
    <dl class="row">
        <dt class="col-sm-2">ID</dt>
        <dd class="col-sm-10">@Model.Id</dd>

        <dt class="col-sm-2">Price</dt>
        <dd class="col-sm-10">@Model.Price.ToString("C")</dd>

        <dt class="col-sm-2">Description</dt>
        <dd class="col-sm-10">@Model.Description</dd>
    </dl>
</div>

<div>
    <a asp-action="Index">Back to List</a>
</div>
```

#### Step 4: Test the Application

Run the app and navigate to `/products`. You'll see the product list. Click "Details" for a product to see its details.

This simple example demonstrates:
- Controller actions returning views with models.
- Routing using conventional defaults (`/products`, `/products/details/1`).
- Razor syntax with `@foreach` and Tag Helpers (`asp-action`, `asp-route-id`).
- Model binding – the `id` parameter is automatically bound from the route.

---

### Summary

You've taken your first major step into ASP.NET Core MVC. You learned:
- The MVC pattern and its separation of concerns.
- How to create a new MVC project and understand its structure.
- Routing—both conventional and attribute—and how URLs map to actions.
- Controllers as the core of request handling, with various action results.
- Views and Razor syntax for dynamic HTML generation, including layouts, partials, and Tag Helpers.

In the next chapter, **"Building Interactive Web Pages with Razor,"** we'll dive deeper into Razor features: layout pages, partial views, view components, and more advanced Tag Helpers. You'll learn how to create reusable UI components and build richer, more maintainable views.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='2. understanding_http_and_the_web.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='4. building_interactive_web_pages_with_razor.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
